This notebook was put together by Balachandar Arumugam. Source and license info is on GitHub.

# Twitter: Discovering Social Circles in Ego Networks

Social networking sites like Facebook, Twitter, LinkedIn etc. offer us many ways to access real-time information from around the world. As our network grows, the information tends to get more and more cluttered, making it difficult to separate signal from the noise. While these social-sites offer ways to categorize friends into groups, the process is very repetitive and also misses out the social-structure present in the network. 

In this notebook, we look into the first degree social-network of the given-user and explore the social-structure within the ego-network. i.e. We look at the given-user's friends and study the network between his friends. Given user is called as 'ego' and his friends are 'alters' and objective is to identify communities/clusters in given user's ego network. 

This notebook covers the following: (1) Connect to Twitter API and build ego-network (network of connections between given user's friends) (2) Identify top-influencers in the ego-network by running PageRank algorithm on Markov Transition matrix (3) Explore the network-structure with help of clustering algorithms to identify clusters of interest (4) Recommend Who-to-Follow on the basis of meaningful Clusters from previous step (5) Visualization: Visualize clusters in 2 dimensions and look for actionable surprises (Using Principal Component Analsyis, multi-dimensional data is reduced to 2 dimensions)

# Collecting Data |  Twitter API

While Twitter API page (https://dev.twitter.com/overview/api/twitter-libraries) recommends many wrappers  I recommend python-twitter

$ pip install python-twitter

Twitter REST APIs are rate-limited (More details here: https://dev.twitter.com/overview/documentation), so I have added a timer in the code to stay within rate-limits). While building the ego-network, there might be cases where someone follows > 5000 Twitter users. I advise modifying wrapper-code to skip to next-node after mining 5000 Ids(a small approximation, to reduce running-time)

Hint: run the 'path' command in notebook find Twitter-wrapper installation-path, and edit the wrapper-method in source-code

In [1]:
from collections import Counter, defaultdict
from sklearn.decomposition import PCA

import csv
import matplotlib.pyplot as plt
import numpy as np
import os.path
import pandas as pd
import seaborn as sns;  sns.set()
import time
import twitter

% matplotlib inline

In [2]:
twitter.__path__  # python-wrapper source code

['C:\\Anaconda\\lib\\site-packages\\twitter']

In [3]:
# initialize file_names 

self_screen_name = 'bala_io'                               # User_of_interest. 'ego' node

fof_filename = self_screen_name + "_friends.csv"           # 'alters' and their follow-structure [FOF]
binaryMap_filename  =  self_screen_name + "_binaryMap.csv" # list of adjacencies as 0 | 1

cache_filename = self_screen_name + "_cache.csv"           # local cache of screen-names, names
cluster_filename = self_screen_name + "_clusters.csv"      # clusters identified

In [4]:
# Twitter API auth | https://dev.twitter.com/oauth/overview/application-owner-access-tokens
# Copy auth details in text-file in same folder as this notebook

with open("credentials.txt", "r") as f:
    reader = csv.reader(f )
    login_dict = {line[0]: line[1]                       
                    for line in reader}        

api = twitter.Api(consumer_key=login_dict.get('consumer_key') ,
                  consumer_secret=login_dict.get('consumer_secret'),
                  access_token_key=login_dict.get('access_token_key'),
                  access_token_secret=login_dict.get('access_token_secret'))
api

In [5]:
# 'ego' and his 'alters'

self_user =  api.GetUser(screen_name = self_screen_name)    # details of 'ego' as a Twitter object
self_user_id = unicode(self_user.id)                        # twitter-id of 'ego'
friends_of_self = api.GetFriendIDs(user_id = self_user_id, screen_name = self_screen_name , stringify_ids = True)
index = [self_user_id] + friends_of_self                    # index for ego-network

In [6]:
# takes fof_filename and to_fetch_list (list of Twitter Ids) as argument, 
# queries Twitter API for 'Following' Ids and writes to local-file, returns None

def update_FoF_File(fileName, to_fetch_list):
    
    with open(fileName, 'a') as f:
        apiRequestCounter = 0
        
        for x in to_fetch_list:
            friends_of_x = api.GetFriendIDs(user_id = x,  stringify_ids = True)  #Twitter API call
            row =  ','.join( [x] +  friends_of_x ) + "\n"            
            f.write(row) 
            
            apiRequestCounter += 1
            if (apiRequestCounter == 14): time.sleep(15*60)  #this API call is rate-limited at 15 req / 15 min

In [7]:
# parses friends_of_friends file and returns list of people, for whom friends' Ids have already been mined

def getFinishList(fileName):
        
    if not os.path.isfile(fileName):
        return [] 
    with open(fileName, 'r') as f:
        return [ x.strip().split(',')[0] for x in f ]  # for every row, 1st entry is a user, followed by his/her friends   

In [8]:
# find the diff, and if any node in 'alters' misses graph-details, fetch that

fof_finish_list = getFinishList( fof_filename )        
fof_to_fetch_list = list ( set(friends_of_self) - set(fof_finish_list) )  # list of nodes, for which fof details are to be fetched
fof_to_fetch_list

[]

In [9]:
update_FoF_File(fof_filename, fof_to_fetch_list) # populate their details in fof_file

In [10]:
# Ego-network as adjacency-matrix
# parses fof_file in order of index, create list of adjacencies as 0 | 1
# returns adjacency-matrix in Row_follows_Column format

def updateBinaryMapFile(fof_filename, binaryMap_filename, index):
    
    with open(fof_filename, "r") as f:
        reader = csv.reader(f)
        fof_dict = {line[0]: line[1:]                        # dict of node:his_followers (excluding 'ego')
                    for line in reader if line[0] in index}
        fof_dict[self_user_id] = index                       # add ego's details at end
    
    bool_list = []
    
    for user in index:
        user_friends = set( fof_dict[user] )  
        bool_row = [item in user_friends for item in index]  # for every node, populate row with T/F 
        bool_list.append(bool_row)
    
    int_nparray = np.array(bool_list) + 0                    # Numpy arithmetic to change boolean to int

    binaryMap_rfc = pd.DataFrame(data = int_nparray, columns= index, index = index)
    binaryMap_rfc.to_csv(binaryMap_filename)
    return binaryMap_rfc                                     # binary map in row_follows_column (rfc) format

In [11]:
binaryMap_rfc = updateBinaryMapFile(fof_filename, binaryMap_filename, index)
binaryMap_rfc.shape

(403, 403)

In [12]:
# Edge-case: Nodes with few outlinks might lead to dead-ends
# To prevent such PageRank leaks, add random outlinks. refer: http://infolab.stanford.edu/~ullman/mmds/ch5.pdf

outlinks_count = binaryMap_rfc.sum(axis = 1)   # horizontal-sum to count outlinks
inlinks_count = binaryMap_rfc.sum(axis = 0)   # vertical-sum to count inlinks

edge_case_nodes =  outlinks_count < 5
edge_case_nodes = edge_case_nodes[edge_case_nodes == True].index

for node in np.array(edge_case_nodes):
    # add 5 random-outlinks
    random_outlinks             = np.random.choice(2, len(index), p=[1-5.0/len(index), 5.0/len(index)]) 
    binaryMap_rfc.loc[node, ] = binaryMap_rfc.loc[node, ] |  random_outlinks

In [13]:
binaryMap_cfr = binaryMap_rfc.transpose()                                # transpose to get in column-follows-row format
colStochMatrix = np.matrix( binaryMap_cfr / binaryMap_cfr.sum(axis = 0)) # column-stochastic-matrix

pageRankVector = np.matrix([1.0/len(index)] *  len(index))               # iniitialize page-rank-vector 
pageRankVector = pageRankVector.transpose()                              # transpose to column-vector

In [14]:
# PageRank algo: Power Iteration to solve Markov transition matrix 
# refer this     : http://setosa.io/blog/2014/07/26/markov-chains/index.html

beta = 0.8
epsilon = 999
while epsilon > (1.0/(10**15)):
    pageRankVectorUpdating = colStochMatrix * pageRankVector * beta # Random teleport happens with 20% prob
    
    # re-insert leaked page-ranks
    S = np.array(pageRankVectorUpdating).sum()                      
    pageRankVectorUpdated = pageRankVectorUpdating + (1 - S) * (1.0/len(index)) * np.ones_like(len(index))
    
    # compute the squared-difference and check for convergence
    error = np.array(pageRankVectorUpdated - pageRankVector)
    epsilon = np.sqrt((error *  error).sum())        
        
    pageRankVector = pageRankVectorUpdated    

In [15]:
# fetches Twitter User_objects for given list of ids, update local_cache to avoid repeat API calls
# returns list of screen_names and names, in same order

def lookup_in_cache(friendsIdsList):

    old_cache, new_cache = pd.DataFrame(), pd.DataFrame()
    screenNamesList, namesList = [], []
    
    if os.path.isfile(cache_filename):
        old_cache = pd.read_csv(cache_filename, dtype=unicode)    # retrieve info if present in local-copy
        to_fetch_list = list ( set (friendsIdsList) - set(old_cache['Ids']) )
    else :        
        to_fetch_list = friendsIdsList   
    
    i = 0
    while (i < len(to_fetch_list) * 1./100):        
        low, high = i * 100, min( len(to_fetch_list), (i+1)*100 )  # UsersLookup api: limit is 100 Ids per request
        twitterObjectsList = api.UsersLookup(user_id = to_fetch_list[low:high])        
        screenNamesList += [unicode(tempObject.screen_name) for tempObject in twitterObjectsList]
        namesList += [tempObject.name.encode('utf-8').strip() for tempObject in twitterObjectsList]
        i = i + 1       
    
    new_cache['screenNames'], new_cache['names'], new_cache['Ids'] = screenNamesList, namesList, to_fetch_list
    new_cache = old_cache.append(new_cache).set_index('Ids', drop = True )
    new_cache.to_csv(cache_filename)    
    
    return list(new_cache.loc[friendsIdsList]['screenNames']), list (new_cache.loc[friendsIdsList]['names'])

In [16]:
# THE data-frame to store output of all computations

clusters_df = pd.DataFrame()
clusters_df['Ids'], clusters_df['PageRanks'] = index, pageRankVector
clusters_df['screenNames'], clusters_df['names'] = lookup_in_cache( list(clusters_df['Ids']) )

clusters_df['Inlinks'], clusters_df['Outlinks'] = list(inlinks_count), list(outlinks_count)
clusters_df = clusters_df.set_index('Ids', drop = True )

n_clusters = min( int( round(np.sqrt(len(index)/2)) ), 10 ) # not more than 10 clusters

In [17]:
# Use K-Means algorithm to cluster friends into different-clusters

from sklearn.cluster import KMeans
est = KMeans(max_iter = 100000, n_clusters = n_clusters, n_init = 250)  
clusters_df['clusterNo'] = est.fit_predict(binaryMap_cfr)

In [18]:
# Use Spectral algorithm to cluster friends into different-clusters

from sklearn import cluster
spectral = cluster.SpectralClustering(n_clusters=10,
                                          eigen_solver='arpack',
                                          affinity="nearest_neighbors")

spectral.fit(binaryMap_cfr)
clusters_df['spectral'] = spectral.labels_.astype(np.int)

C:\Anaconda\lib\site-packages\sklearn\cluster\spectral.py:418: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"


In [19]:
# Use Principal Component Analysis to reduce multiple-dimensions to simple 2 dimensions

pca = PCA(n_components=2)
Xproj = pca.fit_transform(binaryMap_cfr)

clusters_df['dim1'] = Xproj[:,0]
clusters_df['dim2'] = Xproj[:,1]

clusters_df = clusters_df.sort('PageRanks', ascending =False)
clusters_df.to_csv(cluster_filename)

print pca.explained_variance_

[ 10.16897106   2.76655746]


In [20]:
# Overall Top 100 Influencers in Social-Graph 
dummy_df = pd.DataFrame()
for i in range(10):
    dummy_df[i] = list (clusters_df [10*i : 10* i + 10]['names'])

print "*** Top 100 Influencers in my Social Graph *** "
dummy_df

*** Top 100 Influencers in my Social Graph *** 


,0,1,2,3,4,5,6,7,8,9
0,Marc Andreessen,Aaron Levie,Benedict Evans,Hilary Mason,Hunter Walk,mark pincus,David Sacks,Paul Kedrosky,danah boyd,Kevin Weil
1,Elon Musk,Paul Graham,WIRED,Walt Mossberg,Kevin Rose,Horace Dediu,Mike Bloomberg,Steven Sinofsky,Matt Mullenweg,danprimack
2,Tim O'Reilly,Chris Dixon,Chris Anderson,Dave McClure,Josh Kopelman,jason,Peter Fenton,Steven Pinker,Brian Chesky,Paul Buchheit
3,Ev Williams,Ben Horowitz,Nate Silver,Sam Altman,Biz Stone,Tony Fadell,Patrick Collison,Shervin Pishevar,Emily Chang,Chamath Palihapitiya
4,Om Malik,marissamayer,Vinod Khosla,Drew Houston,Bill Gross,Jeff Weiner,Josh Elman,Re/code,Jeff Clavier,Luke Wroblewski
5,Reid Hoffman,Bill Gurley,John Doerr,Tim Cook,Keith Rabois,Jonah Peretti,Ron Conway,Sarah Lacy,Clayton Christensen,Startup L. Jackson
6,Kara Swisher,Chris Sacca,Steven Levy,Nick Bilton,Gabe Rivera,Y Combinator,Jessica Verrilli,Philip Kaplan,Bijan Sabet,Andrew Mason
7,Bill Gates,Eric Schmidt,John Maeda,Joi Ito,Satya Nadella,Naval Ravikant,Pierre Omidyar,dj patil,John Carmack,Dan Frommer
8,Fred Wilson,TechCrunch,Mitch Kapor,Steve Case,Brad Stone,John Markoff,Liz Gannes,Mark Suster,Adam D'Angelo,Balaji S. Srinivasan
9,dick costolo,Max Levchin,Dave Morin,Jessica Lessin,Bradley Horowitz,Bret Taylor,Jeremy Stoppelman,David Lee,Paul Krugman,Kevin Kelly


In [21]:
# KMeans clustering | Top-influencers in Social-Circles
dummy_df = pd.DataFrame()

for i in range(n_clusters):
    
    nodes_in_cluster = list( clusters_df [clusters_df['clusterNo'] == i ]['names'] )     
    if len(nodes_in_cluster) >= 10:            # identify only clusters which are big enough. say size > 20        
        col_name           = str(i) + " : " + str(len(nodes_in_cluster)) + " Ids"
        dummy_df[col_name] = nodes_in_cluster[:10]
        
print "*** KMeans | Cluster-wise Top-influencers [Cluster-number : # of users in the cluster] *** "
dummy_df

*** KMeans | Cluster-wise Top-influencers [Cluster-number : # of users in the cluster] *** 


,1 : 51 Ids,2 : 24 Ids,3 : 23 Ids,4 : 26 Ids,5 : 56 Ids,6 : 47 Ids,7 : 122 Ids,8 : 16 Ids,9 : 31 Ids
0,Nate Silver,WIRED,Tim O'Reilly,Dave Morin,Philip Kaplan,Microsoft Research,Edward Tufte,Marc Andreessen,Mike Bostock
1,Satya Nadella,John Maeda,Bill Gates,Dave McClure,John Carmack,Google Research,Tina Roth Eisenberg,Elon Musk,Hadley Wickham
2,Horace Dediu,John Markoff,marissamayer,Sam Altman,Luke Wroblewski,Lynn Cherny,IDEO,Ev Williams,Drew Conway
3,Tony Fadell,Steven Pinker,Eric Schmidt,Drew Houston,Garry Tan,Andrew Ng,Vivek Wadhwa,Om Malik,Nathan Yau
4,Jonah Peretti,danah boyd,TechCrunch,Jessica Lessin,DHH,Olivier Grisel,Irene Au,Reid Hoffman,Mike Olson
5,Bret Taylor,Clayton Christensen,Chris Anderson,Hunter Walk,Jessica Livingston,Trey Causey,Deb Roy,Kara Swisher,Wes McKinney
6,Mike Bloomberg,Paul Krugman,Vinod Khosla,Josh Kopelman,Sean Ellis,Jure Leskovec,Forbes Tech News,Fred Wilson,Sean J. Taylor
7,Patrick Collison,Kevin Kelly,John Doerr,Keith Rabois,Craig Mod,Yann LeCun,Josh Brewer,dick costolo,Edd Dumbill
8,Jessica Verrilli,steve blank,Steven Levy,Gabe Rivera,Venture Hacks,John D. Cook,Erik Schatzker,Aaron Levie,John Myles White
9,Jeremy Stoppelman,Thomas L. Friedman,Mitch Kapor,mark pincus,Tom Hulme,Fernando Perez,O'Reilly Media,Paul Graham,Amr Awadallah


In [22]:
# Spectral clustering | Top-influencers in Social-Circles

dummy_df = pd.DataFrame()

for i in range(n_clusters):
    
    nodes_in_cluster = list( clusters_df [clusters_df['spectral'] == i ]['names'] )     
    if len(nodes_in_cluster) >= 10:            # identify only clusters which are big enough. say size > 20        
        col_name           = str(i) + " : " + str(len(nodes_in_cluster)) + " Ids"
        dummy_df[col_name] = nodes_in_cluster[:10]
        
print "*** Spectral Clustering | Cluster-wise Top-influencers [Cluster-number : # of users in the cluster] *** "
dummy_df

*** Spectral Clustering | Cluster-wise Top-influencers [Cluster-number : # of users in the cluster] *** 


,0 : 225 Ids,2 : 19 Ids,3 : 17 Ids,4 : 18 Ids,5 : 20 Ids,6 : 19 Ids,7 : 49 Ids,8 : 13 Ids,9 : 17 Ids
0,Tim O'Reilly,Dave McClure,Lynn Cherny,Hilary Mason,Sam Altman,Marc Andreessen,Mike Bostock,Luke Wroblewski,WIRED
1,TechCrunch,Hunter Walk,Olivier Grisel,dj patil,Drew Houston,Elon Musk,Microsoft Research,Daniel Burka,John Maeda
2,Benedict Evans,Josh Kopelman,Trey Causey,Jeff Hammerbacher,Bret Taylor,Ev Williams,Google Research,Josh Brewer,Clayton Christensen
3,Chris Anderson,Keith Rabois,John D. Cook,Hadley Wickham,Patrick Collison,Om Malik,Edd Dumbill,Braden Kowitz,Tom Hulme
4,Nate Silver,Gabe Rivera,Fernando Perez,Drew Conway,Ron Conway,Reid Hoffman,Andrew Ng,Jon Wiley,IDEO
5,Vinod Khosla,mark pincus,Scientific Python,Wes McKinney,Jeremy Stoppelman,Kara Swisher,Jeffrey Heer,Twitter Design,MIT Media Lab
6,John Doerr,jason,Guido van Rossum,Sean J. Taylor,Brian Chesky,Bill Gates,jake hofman,Google Design,Stanford d.school
7,Steven Levy,Naval Ravikant,Jake Vanderplas,Peter Skomoroch,Adam D'Angelo,Fred Wilson,Jure Leskovec,brynn evans,Co.Design
8,Mitch Kapor,David Sacks,Andreas Mueller,John Myles White,Paul Buchheit,dick costolo,Mike Loukides,ZURB,Tim Brown
9,Dave Morin,Peter Fenton,Peter Wang,bradford cross,Andrew Mason,Aaron Levie,Yann LeCun,Jake Knapp,Stanford Business


In [23]:
# takes ids_of_interest, aggregates + counts their friends, and returns who_to_follow list as per requested length

def discover_Friends_toFollow(ids_of_interest, prune_factor = 1, count = 50):    
    
    ids_of_interest  = ids_of_interest[:int(len(ids_of_interest) * prune_factor)]
    print "'who-to-follow' list after looking at:%3d friends' records" %(len(ids_of_interest))
    
    with open(fof_filename) as f:
        reader = csv.reader(f)
        fof_dict = {row[0]:row[0:] for row in reader}  # dict of node:her_followers

    friendsToFollow = []
    for id in ids_of_interest:
        friendsToFollow += list (set(fof_dict[str(id)])  - set(index) ) 

    friendsToFollow = Counter(friendsToFollow).most_common(count)    
    topFriendsIds = [i for i,j in friendsToFollow]
    topFriendsFreq = [j for i,j in friendsToFollow]
           
    topFriendsToFollowDF = pd.DataFrame()
    topFriendsToFollowDF['Ids'], topFriendsToFollowDF['Freq'] = topFriendsIds, topFriendsFreq
    topFriendsToFollowDF['screenNames'], topFriendsToFollowDF['names'] = lookup_in_cache(topFriendsIds)
    
    topFriendsToFollowDF = topFriendsToFollowDF.set_index('Ids', drop = True)          
    return topFriendsToFollowDF    

In [24]:
# 'who_to_follow' on the basis of particular cluster-characterisic
#  e.g. 'hmason' for 'data-science'

clusterNo = clusters_df [ clusters_df['screenNames'] == "hmason" ]['clusterNo']
clusterNo = list(clusterNo)[0]

favorite_cluster_df = clusters_df [clusters_df['clusterNo'] == clusterNo ]
favorite_cluster_list = list(favorite_cluster_df.index)

print "*** Explore the Social-circle of Interest ***"
discover_Friends_toFollow(favorite_cluster_list,1,20)  # recommend 20 Users to follow (who_to_follow)!

*** Explore the Social-circle of Interest ***
'who-to-follow' list after looking at:  7 friends' records


,Freq,screenNames,names
Ids,,,
499497572,7,DCVC,Data Collective
56640837,7,mattocko,Matthew Ocko
50485184,7,zackbogue,Zachary Bogue
14989226,6,daniel_levine,Daniel Levine
18618298,6,flo,Florian Leibert
18367054,6,medriscoll,Michael E. Driscoll
22749476,6,mathieubastian,Mathieu Bastian
14704253,6,mitultiwari,Mitul Tiwari
1497,6,harper,harper


In [25]:
print "*** Explore the complete Ego-Network ***"
discover_Friends_toFollow(clusters_df.index , 0.25, 20) # discover 20 new Users (who_to_follow) after looking at Top 25% influencers in ego-network!

*** Explore the complete Ego-Network ***
'who-to-follow' list after looking at:100 friends' records


,Freq,screenNames,names
Ids,,,
12,71,jack,Jack
9729502,57,travisk,travis kalanick
5017,56,joshu,joshua schachter
5699,55,stewart,Stewart Butterfield
5702,55,Caterina,Caterina Fake
37570179,55,arrington,Michael Arrington
14600116,55,johnbattelle,John Battelle
652193,53,mgsiegler,M.G. Siegler
14202711,53,mattcohler,Matt Cohler
